## Get Pubchem drug feautures

1. finding corresponding Pubchem ids for the drugs 
2. call Pubchem to get chemical properties of the drugs
3. Preprocess text Drug description from the original datasets
4. Preprocess some text characteristics from PubChem properties

In [15]:
import pandas as pd
import numpy as np
import os
# pip install PubChemPy
import pubchempy as pcp
import re
from pubchempy import Compound
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
import pickle
from pubchem_processing import *

_FOLDER = "data/"
_FOLDER_2 = "results/"

In [2]:
os.listdir(_FOLDER)

['drugs_gdsc2.csv',
 'Cell_Line_Features_PANCAN_simple_MOBEM.tsv',
 'Cell_Lines_Details.csv',
 'drugs_gdsc1.csv',
 'Cell_Line_Features_PANCAN_simple_MOBEM.xlsx',
 'fit_no_filt.csv',
 'Drug_Features.csv',
 'normalised_dose_response_data.csv']

In [24]:
drug_features = pd.read_csv(_FOLDER + "Drug_Features.csv").rename(columns={"Drug ID": "DRUG_ID", 
                                                                           "Drug Name": "Drug_Name",
                                                                          "Target Pathway": "Target_Pathway"})
#drug_features.set_index("DRUG_ID", inplace= True)
df = pd.read_csv(_FOLDER+'Drug_Features.csv')
ind_wrong_name = df[df["Drug Name"]=="Lestauritinib"].index
df.loc[ind_wrong_name, "Drug Name"] = "Lestaurtinib"
print(drug_features.shape)
drug_features.head()

(265, 5)


,DRUG_ID,Drug_Name,Synonyms,Target,Target_Pathway
0,1,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,EGFR signaling
1,3,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",MTORC1,PI3K/MTOR signaling
2,5,Sunitinib,"Sutent, Sunitinib Malate, SU-11248","PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",RTK signaling
3,6,PHA-665752,"PHA665752, PHA 665752",MET,RTK signaling
4,9,MG-132,"LLL cpd, MG 132, MG132","Proteasome, CAPN1",Protein stability and degradation


## Part 1: Get PubChem ids

In [4]:
#the most recent data on drugs properties
#https://www.cancerrxgene.org/downloads/anova?screening_set=GDSC1
drugs_1 = pd.read_csv(_FOLDER+'drugs_gdsc1.csv')
drug_features = pd.read_csv(_FOLDER + "Drug_Features.csv").rename(columns={"Drug ID": "DRUG_ID", 
                                                                           "Drug Name": "Drug_Name",
                                                                          "Target Pathway": "Target_Pathway"})

drug_features, not_found = GetPubChemId(drug_features, drugs_1, new_column_name = "pubchem_id")

In [5]:
drug_features[drug_features['pubchem_id'].isnull()]

,DRUG_ID,Drug_Name,Synonyms,Target,Target_Pathway,pubchem_id


In [6]:
drug_features, not_identified_drugs = RunManualCorrections(drug_features, drop_not_found_drugs = False)

Total number of drugs: 250
Number of not found drugs: 17


In [7]:
drug_features[(drug_features["pubchem_id"]=="none")| (drug_features["pubchem_id"]=="several")]

,Drug_Name,Synonyms,Target,Target_Pathway,pubchem_id
DRUG_ID,,,,,
164,JQ12,-,"HDAC1, HDAC2",Chromatin histone acetylation,none
211,TL-2-105,-,not defined,ERK MAPK signaling,none
225,Genentech Cpd 10,-,"AURKA, AURKB",Mitosis,none
231,FMK,KIN001-242,RSK,"Other, kinases",none
253,XMD14-99,-,"ALK, CDK7, LTK, others",Other,none
261,TL-1-85,-,TAK,"Other, kinases",none
286,KIN001-236,-,Angiopoietin-1 receptor,Other,none
330,XMD13-2,-,RIPK1,Apoptosis regulation,none
331,QL-X-138,-,BTK,"Other, kinases",none


In [8]:
drug_features = drug_features[(drug_features["pubchem_id"]!="none")| (drug_features["pubchem_id"]!="none")]
drug_features["pubchem_id"] = drug_features["pubchem_id"].astype("object")

In [9]:
# PubChem_id = 176870
PubChem_id = int(drug_features.loc[1, "pubchem_id"])
CallPubChemOne(PubChem_id)

{'molecular_weight': 393.4,
 'elements': "'C', 'N', 'O', 'H'",
 '2bonds': 8,
 '3bonds': 1,
 'xlogp': 3.3,
 'formal_charge': 0,
 'surface_area': 74.7,
 'complexity': 525,
 'h_bond_donor_count': 1,
 'h_bond_acceptor_count': 7,
 'rotatable_bond_count': 11,
 'heavy_atom_count': 29,
 'atom_stereo_count': 0,
 'defined_atom_stereo_count': 0,
 'undefined_atom_stereo_count': 0,
 'bond_stereo_count': 0,
 'covalent_unit_count': 1,
 'molecular_formula': 'C22H23N3O4',
 'canonical_smiles': 'COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC',
 'inchi_string': 'InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-22-18-13-20(28-10-8-26-2)21(29-11-9-27-3)14-19(18)23-15-24-22/h1,5-7,12-15H,8-11H2,2-3H3,(H,23,24,25)',
 'inchi_key': 'AAKJLRGGTJKAMG-UHFFFAOYSA-N'}

## Part 2 :Getting properties by PubChem API

In [10]:
drug_features, drugs_to_drop = GetPubchemProperties(drug_features)

100%|██████████| 248/248 [03:04<00:00,  1.34it/s]


In [13]:
len(drugs_to_drop)

0

In [16]:
drug_features.drop(drugs_to_drop, axis=0).to_csv(_FOLDER+ "drug_features_raw.csv")

## End of Part 2: Read prepared data

In [23]:
drug_features = pd.read_csv(_FOLDER+ "drug_features_raw.csv").set_index("DRUG_ID")
print(drug_features.shape)

(248, 26)


In [18]:
# pubchem_id is none

drug_features[drug_features["molecular_weight"].isnull()].shape[0]

0

In [19]:
drug_features.columns

Index(['Drug_Name', 'Synonyms', 'Target', 'Target_Pathway', 'pubchem_id',
       'molecular_weight', 'elements', '2bonds', '3bonds', 'xlogp',
       'formal_charge', 'surface_area', 'complexity', 'h_bond_donor_count',
       'h_bond_acceptor_count', 'rotatable_bond_count', 'heavy_atom_count',
       'atom_stereo_count', 'defined_atom_stereo_count',
       'undefined_atom_stereo_count', 'bond_stereo_count',
       'covalent_unit_count', 'molecular_formula', 'canonical_smiles',
       'inchi_string', 'inchi_key'],
      dtype='object')

In [20]:
int_columns = ['2bonds', '3bonds', 'h_bond_donor_count',
       'h_bond_acceptor_count', 'rotatable_bond_count', 'heavy_atom_count',
       'atom_stereo_count', 'defined_atom_stereo_count',
       'undefined_atom_stereo_count', 'bond_stereo_count',
       'covalent_unit_count']
for col in int_columns:
    drug_features[col] = np.int16(drug_features[col])

## Part 3: Preprocessing Text PubChem characteristics

### Presence of some elements (11 elements)

In [21]:
%%time

all_elements = list(set(drug_features["elements"].str.split(",", expand=True).fillna(0).values.flatten())- set([0," 'C'", "'C'", " 'H'"]))
all_elements

elements_in_drugs= list(set([atom.strip(" ").strip("'") for atom in all_elements]))
exceptions =[]
for drug_index in drug_features.index:
    compound_elements = drug_features.loc[drug_index, "elements"]
    print(compound_elements)
    try:
        for i, atom in list(enumerate(elements_in_drugs)):
            if atom in compound_elements:
                drug_features.loc[drug_index, atom] = 1
#                 print(atom, "Yes")
            else:
                drug_features.loc[drug_index, atom] = 0
#                 print(atom, "No")
    except:
        exceptions.append(drug_index)
        drug_features.loc[drug_index, atom] = 0
        
for col in ['B', 'I', 'Br', 'Cl', 'O', 'N', 'F', 'P', 'S', 'Pt']:
    drug_features[col]= np.int16(drug_features[col])
        
print("Exceptions:", drug_features.loc[exceptions, :].shape[0])
print("Elements in drugs:", len(elements_in_drugs), elements_in_drugs)

'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'H'
O No
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl No
Br No
H Yes
'O', 'S', 'N', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F No
Cl Yes
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl Yes
Br No
H Yes
'O', 'S', 'N', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'S', 'N', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F No
Cl Yes
Br No
H

Br No
H Yes
'C', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N No
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'S', 'N', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl Yes
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl Yes
Br No
H Yes
'O', 'S', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F Yes
Cl No
Br No
H Yes
'O', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl

B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'C', 'Br', 'H'
O Yes
Pt No
I No
B Yes
S No
P No
N Yes
F No
Cl No
Br Yes
H Yes
'O', 'S', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F Yes
Cl No
Br No
H Yes
'O', 'S', 'N', 'F', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F Yes
Cl Yes
Br No
H Yes
'O', 'I', 'N', 'F', 'C', 'H'
O Yes
Pt No
I Yes
B No
S No
P No
N Yes
F Yes
Cl No
Br No
H Yes
'O', 'S', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F Yes
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'Cl', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl Yes
Br No
H Yes
'O', 'S', 'N', 'C', 'H'
O Yes
Pt No
I No
B No
S Yes
P No
N Yes
F No
Cl No
Br No
H Yes
'C', 'N', 'O', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F No
Cl No
Br No
H Yes
'O', 'N', 'F', 'C', 'H'
O Yes
Pt No
I No
B No
S No
P No
N Yes
F Yes
Cl No
Br No
H Yes
'O', 'N', 'F', 'Cl', 'C', 'Br', 'H'
O Yes
Pt No
I No
B Yes
S No
P No

In [22]:
drug_features["Br"].value_counts()

0    241
1      7
Name: Br, dtype: int64

In [25]:
drug_features.to_csv(_FOLDER_2 + "drug_features_with_pubchem_properties.csv")

PubChem_features = ["molecular_weight","2bonds", "3bonds", "xlogp", "formal_charge", 
    "surface_area", "complexity", "h_bond_donor_count", 
    "h_bond_acceptor_count", "rotatable_bond_count",
    "heavy_atom_count", "atom_stereo_count", "defined_atom_stereo_count",
    "undefined_atom_stereo_count", "bond_stereo_count", "covalent_unit_count",
    'B', 'I', 'Br', 'Cl', 'O', 'N', 'F', 'P', 'S', 'Pt']

with open(_FOLDER_2 + "X_PubChem_properties.txt", 'w') as f:
    for s in PubChem_features:
        f.write(str(s) + '\n')

print("Number of PubChem features:", len(PubChem_features))

Number of PubChem features: 26


## End of Part 3: Read Prepared Data

In [26]:
drug_features = pd.read_csv(_FOLDER_2+ "drug_features_with_pubchem_properties.csv").set_index("DRUG_ID")

with open(_FOLDER_2 + "X_PubChem_properties.txt", 'r') as f:
    X_PubChem_properties = [line.rstrip('\n') for line in f]

## Part 4: Preprocessing Drugs description from original data

In this section, we are going to have some dumnies columns for Target and Target_Pathway

Converting of Target Pathway resulted in 26 new columns

It is also worth considering elements columns and that deleting columns with C and H which are present in all the compounds

### Dumnies for Target (229) and Target_Pathway (23)

In [44]:
drug_features.head(3)

,Unnamed: 0,Drug_Name,Synonyms,Target,Target_Pathway
DRUG_ID,,,,,
1,0,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,EGFR signaling
3,1,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",MTORC1,PI3K/MTOR signaling
5,2,Sunitinib,"Sutent, Sunitinib Malate, SU-11248","PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",RTK signaling


In [45]:
targets = ""
for x in drug_features["Target"].values:
    targets = targets + ", " + x
targets = list(set(targets.split(", ")[1:]))
print("Number of targets:", len(targets))

Number of targets: 229


In [46]:
df_target = pd.DataFrame(data = np.int32(np.zeros([drug_features.shape[0], len(targets)])), 
                         index = drug_features.index, 
                         columns = targets)
len(targets)

229

In [47]:
for index in drug_features.index:
    targets_i = drug_features.loc[index, "Target"].split(", ")
    df_target.loc[index, targets_i]=1
df_target.shape

(265, 229)

In [48]:
print("Number of unique pathways:", drug_features["Target_Pathway"].nunique())

df_target_target_pathway = pd.concat([df_target, pd.get_dummies(drug_features["Target_Pathway"])], axis=1)
df_target_target_pathway.shape

Number of unique pathways: 23


(265, 252)

In [50]:
drug_features["Target_Pathway"]

DRUG_ID
1                          EGFR signaling
3                     PI3K/MTOR signaling
5                           RTK signaling
6                           RTK signaling
9       Protein stability and degradation
                      ...                
1498                   ERK MAPK signaling
1502                      Hormone-related
1526                   ERK MAPK signaling
1527                  PI3K/MTOR signaling
1529                                Other
Name: Target_Pathway, Length: 265, dtype: object

In [49]:
df_final = pd.concat([drug_features.drop(["Target_Pathway"], axis=1), df_target_target_pathway], axis=1)
df_final.shape

(265, 256)

In [51]:
df_final

,Unnamed: 0,Drug_Name,Synonyms,Target,IRAK1,dsDNA break induction,SGK2,CDK5,Retinoic acid,JAK2,...,JNK and p38 signaling,Metabolism,Mitosis,Other,"Other, kinases",PI3K/MTOR signaling,Protein stability and degradation,RTK signaling,WNT signaling,p53 pathway
DRUG_ID,,,,,,,,,,,,,,,,,,,,,
1,0,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",MTORC1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,2,Sunitinib,"Sutent, Sunitinib Malate, SU-11248","PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,3,PHA-665752,"PHA665752, PHA 665752",MET,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,4,MG-132,"LLL cpd, MG 132, MG132","Proteasome, CAPN1",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,260,Selumetinib,"AZD6244, AZD-6244, ARRY-886","MEK1, MEK2",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1502,261,Bicalutamide,"ICI-176334, Casodex, Cosudex, ICI 176334",AR,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1526,262,Refametinib,"RDEA119, BAY-86-9766, BAY 869766","MEK1, MEK2",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
df_final.to_csv(_FOLDER_2 + "drug_features_with_pubchem_properties_final.csv")

with open(_FOLDER_2 + "X_features_Targets.txt", 'w') as f:
    for s in targets:
        f.write(str(s) + '\n')
        
with open(_FOLDER_2 + "X_features_Target_Pathway.txt", 'w') as f:
    for s in drug_features["Target_Pathway"].unique():
        f.write(str(s) + '\n')   

## End of Part 4: Read Prepared Data

In [120]:
drug_features = pd.read_csv(_FOLDER_2 + "drug_features_with_pubchem_properties_final.csv")

with open(_FOLDER_2 + "X_features_Targets.txt", 'r') as f:
    X_features_Targets = [line.rstrip('\n') for line in f]
    
with open(_FOLDER_2 + "X_features_Target_Pathway.txt", 'r') as f:
    X_features_Target_Pathway = [line.rstrip('\n') for line in f]

In [121]:
drug_features.head()

,DRUG_ID,Drug_Name,Synonyms,Target,molecular_weight,elements,2bonds,3bonds,xlogp,formal_charge,...,JNK and p38 signaling,Metabolism,Mitosis,Other,"Other, kinases",PI3K/MTOR signaling,Protein stability and degradation,RTK signaling,WNT signaling,p53 pathway
0,1,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,393.4,"'H', 'O', 'N', 'C'",8,1,3.3,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",MTORC1,914.2,"'H', 'O', 'N', 'C'",9,0,6.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,5,Sunitinib,"Sutent, Sunitinib Malate, SU-11248","PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",398.5,"'N', 'H', 'C', 'O', 'F'",8,0,2.6,0.0,...,0,0,0,0,0,0,0,1,0,0
3,6,PHA-665752,"PHA665752, PHA 665752",MET,641.6,"'N', 'H', 'S', 'Cl', 'C', 'O'",13,0,5.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,9,MG-132,"LLL cpd, MG 132, MG132","Proteasome, CAPN1",475.6,"'H', 'O', 'N', 'C'",7,0,4.8,0.0,...,0,0,0,0,0,0,1,0,0,0


In [122]:
with open(_FOLDER_2+"X_features_cancer_cell_lines.txt", 'r') as f:
    X_cancer_cell_lines = [line.rstrip('\n') for line in f]

In [123]:
print("Final Features: \n")
print("Cell lines (CCL) features:", len(X_cancer_cell_lines))
print("PubChem drug features:", len(PubChem_features))
print("Drug description features - Targets: %d, Target_Pathway: %d" % (len(X_features_Targets), len(X_features_Target_Pathway)))

Final Features: 

Cell lines (CCL) features: 1073
PubChem drug features: 26
Drug description features - Targets: 212, Target_Pathway: 23


In [124]:
all_elements

["'H'",
 " 'Cl'",
 " 'N'",
 " 'B'",
 " 'I'",
 "'N'",
 " 'P'",
 " 'O'",
 " 'Pt'",
 " 'Br'",
 "'O'",
 " 'F'",
 " 'S'"]